# Trabajo Práctico N°2 - Introducción a Inteligencia Artificial | Regresión del valor de valor medio de casas en distritos de California

Este es un [dataset](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset) muy popular que vamos a leer desde **Scikit-Learn**.

Se requiere construir un modelo de regresión que nos permita predecir el valor medio de las casas en distintos distritos de California (medido en cientos de miles de dólares, es decir, $100,000). Este conjunto de datos proviene del censo de EE. UU. de 1990, donde cada observación corresponde a un bloque. Un bloque es la unidad geográfica más pequeña para la cual la Oficina del Censo de EE. UU. publica datos de muestra (típicamente con una población de entre 600 y 3,000 personas).


Un hogar es un grupo de personas que residen dentro de una misma vivienda. Dado que el número promedio de habitaciones y dormitorios en este conjunto de datos se proporciona por hogar, estas columnas pueden tomar valores altos en bloques con pocos hogares y muchas viviendas vacías.

Los atributos, en el orden en que se guardaron en el dataset, son:

- `MedInc`: Ingreso medio del bloque
- `HouseAge`: Edad mediana de las viviendas en el bloque
- `AveRooms`: Número promedio de habitaciones por hogar
- `AveBedrms`: Número promedio de dormitorios por hogar
- `Population`: Población del bloque
- `AveOccup`: Número promedio de personas por hogar
- `Latitude`: Latitud del bloque
- `Longitude`: Longitud del bloque

### Import de librerías y datos 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LinearRegression, Ridge

sns.set()

In [ ]:
# Lectura del dataset
california_housing = fetch_california_housing()

# Obtenemos los atributos y el target:
X = california_housing.data
y = california_housing.target

# Transformamos los datos a estructuras de Pandas:
X = pd.DataFrame(X, columns=california_housing['feature_names'])
y = pd.Series(y, name=california_housing['target_names'][0])

# Unimos X e y; esto es útil para generar el mapa de calor de correlaciones
df_california = pd.concat([X, y], axis=1)

### Análisis exploratorio de datos (EDA)

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
df_california.head()

In [ ]:
df_california.info()

In [ ]:
df_california.describe()

In [ ]:
plt.figure(figsize=(15, 10))
sns.set_style("white")
sns.set_context("talk")
sns.set_style("ticks")
sns.heatmap(df_california.corr(), annot=True, cmap='coolwarm', annot_kws={"size": 10}, fmt=".2f", cbar=True)
plt.show()

In [ ]:
feature = "MedHouseVal"

corr = df_california.corr()[[feature]].sort_values(by=feature, ascending=False)

plt.figure(figsize=(4, 8))
sns.heatmap(corr, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title(f"Correlación de {feature} con otras variables")
plt.show()

In [ ]:
# Las features mas corelacionadas con el target son: MedInc; AveRooms; y Latitude. 

In [ ]:
# pairplot para correlación entre variables y target
plt.figure(figsize=(15, 10))
sns.pairplot(data=df_california, diag_kind='kde', corner=True)
plt.show()


In [ ]:
# Grafico de histograma de features numéricas
num_cols = df_california.select_dtypes(include='number').columns
n_cols = 3  # cantidad de columnas en la grilla
n_rows = (len(num_cols) + n_cols - 1) // n_cols  # filas necesarias

fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4 * n_rows))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    sns.histplot(df_california[col], bins=30, ax=axes[i], kde=False)
    axes[i].set_title(f'Histograma de {col}')

# Sacar ejes vacíos si hay
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

### División en entrenamiento y evaluación; normalización 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,  y, test_size=0.3, random_state=42)

In [ ]:
# realizamos un heatmap sobre la matriz X resultante, population esta fuera de escala. 
sns.set_style("white")
sns.set_context("talk")
sns.set_style("ticks")
sns.heatmap(X_train)
plt.show()

In [ ]:
# Escalado de los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=california_housing['feature_names'])
X_test_scaled = pd.DataFrame(X_test_scaled, columns=california_housing['feature_names'])

In [ ]:
# realizamos un heatmap sobre la matriz X resultante, datos mas escalados ahora
sns.set_style("white")
sns.set_context("talk")
sns.set_style("ticks")
sns.heatmap(X_train_scaled)
plt.show()

### Modelos: Regresión Lineal y Ridge (con búsqueda de hiperparametro)

In [ ]:
# Seteamos modelo inicial de regresión 

Linear = LinearRegression()
Linear.fit(X_train_scaled, y_train)

scoring = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']

scores = cross_validate(Linear, X_train_scaled, y_train, cv=5, scoring=scoring, return_train_score=False)

print("R2 scores:", scores['test_r2'])
print("MSE scores:", -scores['test_neg_mean_squared_error'])
print("MAE scores:", -scores['test_neg_mean_absolute_error'])

print("\nPromedios:")
print("R2 promedio:", np.mean(scores['test_r2']))
print("MSE promedio:", -np.mean(scores['test_neg_mean_squared_error']))
print("MAE promedio:", -np.mean(scores['test_neg_mean_absolute_error']))

In [ ]:
# Resultados para ejercicio n°3 

y_pred_train = Linear.predict(X_train_scaled)

TSS = ((y_train - y_train.mean())**2).sum() #TSS total sum of squares
RSS = ((y_train - y_pred_train)**2).sum() #RSS residual sum of squares
ESS = ((y_pred_train - y_train.mean())**2).sum() # ESS explained sum of squares

r2_linear = ESS / TSS  # o 1 - RSS / TSS

print("\nComparación de varianza:")
print(f"Varianza total de los datos (TSS): {TSS}")
print(f"Varianza residual (RSS): {RSS}")
print(f"Varianza explicada por el modelo (ESS): {ESS}")
print(f"Coeficiente de determinación (R^2): {r2_linear:.4f}")


In [ ]:
# Rango de alpha
alpha_range = np.linspace(0.01, 12.5, 50)  # 50 valores entre 0.01 y 12.5

# Modelo Ridge
ridge = Ridge()

# Definir scoring (MSE negativo porque sklearn maximiza la métrica)
scorer = make_scorer(mean_squared_error, greater_is_better=False)

# GridSearch con CV de 5 folds
param_grid = {'alpha': alpha_range}
grid = GridSearchCV(ridge, param_grid, cv=5, scoring=scorer)
grid.fit(X_train_scaled, y_train)

# Extraer resultados
mean_scores_ridge = -grid.cv_results_['mean_test_score']  # convertir a positivo
best_alpha = grid.best_params_['alpha']

print(f"Mejor alpha: {best_alpha}")
print(f"MSE promedio con mejor alpha: {mean_scores_ridge[grid.best_index_]}")

best_ridge = grid.best_estimator_

In [ ]:
# Graficar MSE vs alpha
plt.figure(figsize=(8,5))
plt.plot(alpha_range, mean_scores_ridge, marker='o')
plt.xlabel("Alpha")
plt.ylabel("MSE (validación cruzada)")
plt.title("Ridge Regression: MSE vs Alpha (5-fold CV)")
plt.axvline(best_alpha, color='r', linestyle='--', label=f'Mejor alpha = {best_alpha:.2f}')
plt.legend()
plt.grid(True)
plt.show()

### Resultados y conclusiones TP

#### Ejecución de modelos

In [ ]:
# Validacion de resultados contra validation set 

# Baseline

baseline = y_train.mean()
y_pred_baseline = np.full(len(y_test), baseline)

r2_score_baseline = r2_score(y_test, y_pred_baseline)
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)
mse_baseline = mean_squared_error(y_test, y_pred_baseline)

print("\nResultados para Baseline (media):")
print("R²:", r2_score_baseline)
print("MSE:", mse_baseline)
print("MAE:", mae_baseline)

In [ ]:
# Validacion de resultados contra validation set 

# Linear

y_pred_linear = Linear.predict(X_test_scaled)

r2_score_linear = r2_score(y_test, y_pred_linear)
mae_linear = mean_absolute_error(y_test, y_pred_linear)
mse_linear = mean_squared_error(y_test, y_pred_linear)

variance_linear = np.var(y_test - y_pred_linear)

print("\nResultados para regresión lineal:")
print("R²:", r2_score_linear)
print("MSE:", mse_linear)
print("MAE:", mae_linear)


In [ ]:
# Validacion de resultados contra validation set 

# Ridge

y_pred_ridge = best_ridge.predict(X_test_scaled)

r2_score_ridge = r2_score(y_test, y_pred_ridge)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)

print("\nResultados para regresión Ridge:")
print("R²:", r2_score_ridge)
print("MSE:", mse_ridge)
print("MAE:", mae_ridge)


#### Resultados 

In [ ]:
# Ejercicio n°3: comparación de resultados de varianza de modelo vs datos. 
print("\nComparación de varianza")
print("Varianza de los datos:", np.var(y_test))
print("Varianza del modelo lineal:", variance_linear)
print("Coeficiente de Pearson (R2) para la regresión lineal:", r2_score_linear)


In [ ]:

# Formulación de tabla comparativa de resultados

results = {
    'Model': ['Baseline', 'Linear Regression', f'Ridge (α={best_alpha:.2f})'],
    'R²': [
        r2_score_baseline,
        r2_score_linear,
        r2_score_ridge
    ],
    'MSE': [
        mse_baseline,
        mse_linear,
        mse_ridge
    ],
    'MAE': [
        mae_baseline,
        mae_linear,
        mae_ridge
    ]
}

# Crear DataFrame
df_results = pd.DataFrame(results)

# Mostrar tabla con formato
styled = (
    df_results.style
    .format({'R²': '{:.4f}', 'MSE': '{:.4f}', 'MAE': '{:.4f}'})
    .set_properties(**{'text-align': 'center'})
)

styled

In [ ]:
# Resultados son bastante mediocres, se podría intentar retirar outliers de MedHouseVal

### Propuesta de mejora de modelo 


In [ ]:
# datos de medhouseval 
df_california['MedHouseVal'].describe()

In [ ]:
df_california["MedHouseVal"].plot(kind="box")
plt.title("Boxplot de MedHouseVal")
plt.show()

In [ ]:
# Separación de outliers 
col = df_california["MedHouseVal"]
Q1 = col.quantile(0.25)
Q3 = col.quantile(0.75)
IQR = Q3 - Q1

#limites
limite_superior = Q3 + 1.5 * IQR
limite_inferior = Q1 - 1.5 * IQR

df_california_filtrada = df_california[(col >= limite_inferior) & (col <= limite_superior)]

In [ ]:
print("Cantidad con outliers:", len(df_california))
print("Cantidad sin outliers:", len(df_california_filtrada))

In [ ]:
X_new = df_california_filtrada.drop(columns=['MedHouseVal'])
y_new = df_california_filtrada['MedHouseVal']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_new,  y_new, test_size=0.3, random_state=42)

In [ ]:
# Escalado de los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=california_housing['feature_names'])
X_test_scaled = pd.DataFrame(X_test_scaled, columns=california_housing['feature_names'])

In [ ]:
# Seteamos modelo inicial de regresión 

Linear = LinearRegression()
Linear.fit(X_train_scaled, y_train)

scoring = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']

scores = cross_validate(Linear, X_train_scaled, y_train, cv=5, scoring=scoring, return_train_score=False)

print("R2 scores:", scores['test_r2'])
print("MSE scores:", -scores['test_neg_mean_squared_error'])
print("MAE scores:", -scores['test_neg_mean_absolute_error'])

print("\nPromedios:")
print("R2 promedio:", np.mean(scores['test_r2']))
print("MSE promedio:", -np.mean(scores['test_neg_mean_squared_error']))
print("MAE promedio:", -np.mean(scores['test_neg_mean_absolute_error']))

In [ ]:
# Rango de alpha
alpha_range = np.linspace(0.01, 12.5, 50)  # 50 valores entre 0.01 y 12.5

# Modelo Ridge
ridge = Ridge()

# Definir scoring (MSE negativo porque sklearn maximiza la métrica)
scorer = make_scorer(mean_squared_error, greater_is_better=False)

# GridSearch con CV de 5 folds
param_grid = {'alpha': alpha_range}
grid = GridSearchCV(ridge, param_grid, cv=5, scoring=scorer)
grid.fit(X_train_scaled, y_train)

# Extraer resultados
mean_scores_ridge = -grid.cv_results_['mean_test_score']  # convertir a positivo
best_alpha_2 = grid.best_params_['alpha']

print(f"Mejor alpha: {best_alpha_2}")
print(f"MSE promedio con mejor alpha: {mean_scores_ridge[grid.best_index_]}")

best_ridge = grid.best_estimator_

In [ ]:
# Validacion de resultados contra validation set 

# Linear

y_pred_linear = Linear.predict(X_test_scaled)

r2_score_linear_2 = r2_score(y_test, y_pred_linear)
mae_linear_2 = mean_absolute_error(y_test, y_pred_linear)
mse_linear_2 = mean_squared_error(y_test, y_pred_linear)


In [ ]:
# Validacion de resultados contra validation set 

# Ridge

y_pred_ridge = best_ridge.predict(X_test_scaled)

r2_score_ridge_2 = r2_score(y_test, y_pred_ridge)
mae_ridge_2 = mean_absolute_error(y_test, y_pred_ridge)
mse_ridge_2 = mean_squared_error(y_test, y_pred_ridge)


In [ ]:

# Formulación de tabla comparativa de resultados

results = {
    'Model': ['Baseline', 'Linear Regression', f'Ridge (α={best_alpha:.2f})', 'Linear Regression (Outliers Removed)', f'Ridge (Outliers removed, α={best_alpha_2:.2f})'],
    'R²': [
        r2_score_baseline,
        r2_score_linear,
        r2_score_ridge,
        r2_score_linear_2,
        r2_score_ridge_2
    ],
    'MSE': [
        mse_baseline,
        mse_linear,
        mse_ridge,
        mse_linear_2,
        mse_ridge_2
    ],
    'MAE': [
        mae_baseline,
        mae_linear,
        mae_ridge,
        mae_linear_2,
        mae_ridge_2
    ]
}

# Crear DataFrame
df_results = pd.DataFrame(results)

# Mostrar tabla con formato
styled = (
    df_results.style
    .format({'R²': '{:.4f}', 'MSE': '{:.4f}', 'MAE': '{:.4f}'})
    .set_properties(**{'text-align': 'center'})
)

styled

In [ ]:
# Modelo mejora 0.06 en R2 y reduce MSE en alrededor de 0.2, asi como 0.06 en MAE. 